# LAB 3.1 — Ước lượng hình học làn đường từ mặt nạ Bird’s-Eye-View
ACE Lane-Keeping System – Geometry Module  

## 1. Overview
Lab này trình bày **các phép tính hình học cốt lõi** dùng để chuyển đổi một mặt nạ làn đường nhị phân (từ BEV) thành:

- Độ lệch ngang của xe (`pos_m`)
- Sai lệch góc hướng (`head_deg`)
- Lấy mẫu đa tỉ lệ trên làn đường
- Chọn thích ứng tỉ lệ đáng tin cậy nhất

Các đầu ra này là **đầu vào cho Adaptive Lane Controller** được sử dụng trong LAB 3.3.

## 2. Learning Objectives

Sau lab này, người học sẽ hiểu:

1. Cách tính tâm làn đường từ mặt nạ BEV  
2. Cách ước lượng góc hướng bằng hai điểm lấy mẫu theo phương dọc  
3. Cách chuyển đổi độ rộng pixel → mét thực tế  
3. Cách multi-ratio cải thiện độ ổn định  
4. Cách đánh giá độ ổn định bằng thống kê cửa sổ  

## 3. Import Libraries


In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
from collections import deque

## 4. Tải BEV Mask (Input from LAB 2)

Trong triển khai thực tế, mặt nạ này được tạo ra từ:

mask01 → ROI → morphology → BEV

Trong lab này, chúng ta tải một BEV mask nhị phân mẫu.


In [ ]:
bev = cv2.imread(r"C:\Users\admin\ACE_Finalv3\debug_pipeline_out\frame_00600_4_bev.png", cv2.IMREAD_GRAYSCALE)
bev = (bev > 0).astype(np.uint8)

In [ ]:
plt.imshow(bev, cmap='gray')
plt.title("BEV Mask")
plt.axis('off')
plt.show()

## 5. Tính hệ số tỷ lệ độ rộng làn (meters_per_pixel)

Để chuyển đổi khoảng cách theo pixel sang mét, chúng ta ước lượng độ rộng làn từ hàng dưới cùng:

$$
meters\_per\_pixel = \frac{lane\_width\_m}{lane\_width\_px}
$$

Chúng ta giả định độ rộng làn thực tế là **0.20 m** (theo thông số Autocar-Kit).


In [ ]:
def meters_per_pixel(bev, lane_width_m=0.20):
    """
    Estimate meters-per-pixel (mpp) at the bottom row of BEV mask.
    - bev: binary BEV mask
    - lane_width_m: real-world lane width in meters (default 0.20m for minicar)
    """

    H, W = bev.shape
    row = bev[H - 1, :]
    xs = np.where(row > 0)[0]        # lane pixel indices


    if xs.size >= 2:
        # TODO: tính chiều rộng lane theo pixel
        ### YOUR CODE HERE ###
        # width_px = ?

        # TODO: chống nhiễu nếu width quá nhỏ (<16 px)
        ### YOUR CODE HERE ###
        # width_px = max(width_px, ...)

        # TODO: đổi sang m/pixel
        ### YOUR CODE HERE ###
        # mpp = ?

        return mpp

    # Không phát hiện lane → fallback
    return lane_width_m / max(W, 1)


## 6. Tính vị trí tâm làn đường (center_x)

Sau khi tính được hệ số tỷ lệ độ rộng làn (meters-per-pixel), bước tiếp theo là
đo **tâm ngang của làn đường** tại một độ cao nhất định trong ảnh BEV.

Với một tỷ lệ ( r trong (0,1) ):

$$
y = r \cdot H
$$

Chúng ta quét mặt nạ BEV tại hàng \( y \) và tìm các pixel làn đường:

$$
x_{\text{lane}} = \{\, x \mid bev[y, x] = 1 \,\}
$$

Nếu tồn tại ít nhất hai pixel làn đường:

$$
x_{\text{center}} = \frac{x_{\min} + x_{\max}}{2}
$$

Ngược lại, chúng ta trả về **NaN**, cho biết làn đường không được nhìn thấy tại độ cao này.


In [ ]:
def center_x(bev, ratio):
    """
    Compute the horizontal center of the lane at a given height ratio.
    
    Parameters:
    - bev: binary BEV mask (0/1)
    - ratio: vertical position ratio (0 = top, 1 = bottom)
    
    Returns:
    - cx: lane center x-coordinate (float), or NaN if lane is not visible
    """

    H, W = bev.shape
    # Convert ratio to a valid row index
    y = int(ratio * H)
    y = np.clip(y, 0, H - 1)
    # Find lane pixels in this row
    xs = np.where(bev[y, :] > 0)[0]

    if xs.size < 2:
        return np.nan 

    # TODO 1: compute lane center x-coordinate
    ### YOUR CODE HERE ###
    # cx = 

    return cx


## 7. Tính độ lệch ngang của làn (posₘ)

Sau khi có:

1. **Tâm làn đường** $( x_{\text{center}} $) từ Cell 6  
2. **Meters-per-pixel** $( mpp $) từ Cell 5  

chúng ta có thể tính **độ lệch ngang** của xe so với tâm làn.

Giả sử độ rộng ảnh BEV là \( W \).  
Tâm tham chiếu của xe là:

$$
x_{\text{ref}} = \frac{W}{2}
$$

Độ lệch ngang theo mét là:

$$
pos_m = (x_{\text{ref}} - x_{\text{center}})\,\times\, mpp
$$

Diễn giải:
- $( pos_m > 0 $): xe đang **lệch trái** so với tâm → cần đánh lái **sang phải**  
- $( pos_m < 0 $): xe đang **lệch phải** so với tâm → cần đánh lái **sang trái**  

Nếu không thể phát hiện tâm làn đường, trả về `NaN`.


In [ ]:
def compute_lateral_offset(bev, ratio, mpp):
    """
    Compute lateral offset (pos_m) using:
        - BEV mask
        - center ratio (0–1)
        - meters-per-pixel (mpp)

    Returns:
        pos_m (float), or NaN if lane not detected.
    """

    H, W = bev.shape
    # Step 1: get lane center at this height
    cx = center_x(bev, ratio)
    if np.isnan(cx):
        return np.nan
    # Step 2: compute reference vehicle center
    ### YOUR CODE HERE ###
    # x_ref = ?

    # Step 3: compute lateral offset pos_m
    ### YOUR CODE HERE ###
    # pos_m = (x_ref - cx) * mpp

    return pos_m


## 8. Tính góc hướng (θ) từ hình học BEV

Để ước lượng hướng đánh lái, chúng ta tính **góc hướng** của làn dựa trên
hai điểm được lấy mẫu tại các độ cao khác nhau của mặt nạ BEV.

Gọi:

- $( y_b $) = hàng lấy mẫu phía dưới  
- $( y_t $) = hàng lấy mẫu phía trên  
- $( x_b $) = tâm làn tại $( y_b $)  
- $( x_t $) = tâm làn tại $( y_t $)

Ta định nghĩa:

$$
dx = x_b - x_t, \qquad dy = y_b - y_t
$$

Góc hướng theo độ là:

$$
\theta = \arctan2(dx,\, dy) \cdot \frac{180}{\pi}
$$

Diễn giải:

- $( \theta > 0 $): làn **cong trái**  
- $( \theta < 0 $): làn **cong phải**  
- Nếu thiếu một trong hai tâm làn → trả về `NaN`

Đây là công thức hình học được sử dụng trong các nghiên cứu lane detection phổ biến
và cũng chính là công thức được áp dụng trong pipeline đầy đủ của bạn.


In [ ]:
def heading_deg_at_ratio(bev, y_ratio, dy_px=30):

    """
    Compute heading angle from BEV using two sampling heights.
    
    Inputs:
        bev: BEV binary mask
        y_ratio: vị trí mẫu (0 = đỉnh, 1 = đáy BEV)
        dy_px: khoảng cách theo trục dọc (px)
    Returns:
        heading_deg (float) or NaN,  xb : float
    """
    H, W = bev.shape
    # 1. Convert ratios → pixel coordinates
    yb = int(np.clip(y_ratio * H, 0, H - 1))
    yt = int(np.clip(yb - dy_px, 0, H - 1))
    # 2. Compute lane centers at the two heights
    xb = center_x(bev, yb / H)    # center at bottom position
    xt = center_x(bev, yt / H)    # center at upper position
    # TODO: compute dx, dy
    ### YOUR CODE HERE ###
    # dx = ?
    # dy = ?

    # TODO: compute heading angle in degrees using arctan2
    ### YOUR CODE HERE ###
    # heading_deg = ?

    return heading_deg, xb



## 9. Multi-Ratio Sampling (Trích xuất nhiều đo đạc dọc BEV)

Một điểm đo duy nhất (single-ratio) thường không đủ ổn định, đặc biệt khi:
- lane bị nhiễu
- lane bị mất 1 phần
- góc cua gắt

Giải pháp: **Lấy nhiều mẫu tại nhiều vị trí khác nhau trong ảnh BEV** (multi-ratio sampling).

Cho tập tỷ lệ mẫu:

$$
R = \{ r_1, r_2, r_3, \ldots \}
$$

Tại mỗi tỷ lệ $( r_i $):

1. Lấy tâm lane:
$$
x_i = center\_x(BEV, r_i)
$$

2. Tính lệch tâm:
$$
pos_i = (W/2 - x_i) \cdot mpp
$$

3. Tính góc hướng:
$$
head_i = heading(r_i)
$$

Kết quả: danh sách `pos_list`, `head_list`, `center_list` dùng cho adaptive controller.


In [ ]:
def multi_ratio_measure(bev, ratios, dy_px=30, lane_width_m=0.20):
    """
    Perform multi-height lane measurements on BEV mask.

    Inputs:
        bev: BEV mask (0/1)
        ratios: list of sampling ratios (e.g., [0.98, 0.92, 0.82, 0.72])
        dy_px: vertical offset for heading calculation
        lane_width_m: physical lane width (20 cm for Autocar-Kit)

    Returns:
        centers_px: list of center-x positions at each ratio
        pos_list:   lateral offset (meters)
        head_list:  heading angle (degrees)
    """

    H, W = bev.shape

    # Meters-per-pixel scaling
    mpp = meters_per_pixel(bev, lane_width_m=lane_width_m)

    centers_px = []
    pos_list   = []
    head_list  = []

    for r in ratios:

        # TODO: Tính tâm lane theo pixel
        ### YOUR CODE HERE ###
        # cx =center_x(bev, r)

        # TODO: Tính heading angle
        ### YOUR CODE HERE ###
        # head_deg, _ = 

        # TODO: Tính độ lệch pos (m)
        ### YOUR CODE HERE ###
        # pos_m = 

        centers_px.append(cx)
        pos_list.append(pos_m)
        head_list.append(head_deg)

    return centers_px, pos_list, head_list


In [ ]:
ratios = [0.98, 0.92, 0.82, 0.72]

centers_px, pos_list, head_list = multi_ratio_measure(
    bev, ratios=ratios, dy_px=30, lane_width_m=0.20
)

assert len(centers_px) == len(ratios)
assert len(pos_list)   == len(ratios)
assert len(head_list)  == len(ratios)

print("Multi-ratio measurement PASS ✔")
centers_px, pos_list, head_list

In [ ]:
H = bev.shape[0]

plt.figure(figsize=(6,6))
plt.imshow(bev * 255, cmap="gray", origin="upper")

for i, cx in enumerate(centers_px):
    if not np.isnan(cx):
        y = int(ratios[i] * H)
        plt.scatter(cx, y, s=60, label=f"r={ratios[i]:.2f}")

plt.gca().invert_yaxis()
plt.title("Multi-Ratio Lane Sampling (BEV)")
plt.legend()
plt.show()


## 10. Adaptive Ratio Selection

Trong Cell Multi-Ratio trước, chúng ta đã trích xuất:
- `centers_px[r]` — tâm lane theo pixel
- `pos_list[r]` — độ lệch tâm theo mét
- `head_list[r]` — góc hướng của làn

Nhưng mỗi tỷ lệ $( r $) có độ tin cậy khác nhau:

- Có tỷ lệ nhìn thấy lane rõ → center ổn định - Có tỷ lệ bị nhiễu hoặc lane đứt đoạn → kết quả dao động mạnh
- Một số vị trí có thể bị mất lane tạm thời

Do đó, ta cần thuật toán **Adaptive Selection** để tự động chọn tỷ lệ tốt nhất mỗi frame.



### 1) Các đặc trưng đánh giá độ ổn định

Cho mỗi tỷ lệ $( r $):

#### ● Độ dao động của pos
$$
std\_pos = \mathrm{STD}(pos[r])
$$
Dao động càng thấp → tín hiệu càng ổn định sẽ được ưu tiên

#### ● Độ lệch trung bình của head
$$
mean\_head = \mathrm{MEAN}(head[r])
$$
Giá trị tuyệt đối càng lớn → có khả năng bị nhiễu hoặc đánh giá sai hình dạng cua.

#### ● Mức độ quan sát (coverage)
$$
coverage = \mathrm{MEAN}(valid[r])
$$
Đây là tỷ lệ số frame nhìn thấy lane tại vị trí đó.



### 2) Ý nghĩa các hệ số W_STAB, W_HEAD, W_LAT, EPS

Đây là **các trọng số dùng để cân bằng tiêu chí ổn định – độ tin cậy – vị trí**.

#####       `W_STAB` — trọng số ưu tiên độ ổn định
Giá trị lớn (ví dụ 2.0) cho thấy ta ưu tiên **pos ổn định**, tránh rung hoặc nhảy tâm đột ngột.
- Lý do: Ưu tiên các điểm ratio giúp xe độ ổn định để bám làn

#####       `W_HEAD` — phạt độ lệch hướng lớn
Giá trị nhỏ (ví dụ 0.1) vì góc hướng rất nhạy, không nên phạt quá mạnh.
- Lý do: head có thể biến động tự nhiên khi vào cua → không nên trừng phạt quá mức.

#####       `W_LAT` — phạt các tỷ lệ r quá cao (vùng xa)
Giá trị nhỏ (0.05) dùng để tránh chọn vùng quá xa trên BEV.
- Lý do: Ưu tiên cho các điểm gần với xe để xe có quỹ đạo ổn định nhất

#####       `EPS` — hệ số chống chia cho 0
$$
std\_pos + \epsilon
$$
Giá trị rất nhỏ (1e-6) để tránh chia 0 khi std_pos = 0 (vô cùng ổn định).



### 3) Công thức chấm điểm tổng hợp

$$
score(r) = 
coverage \left(
\frac{W_{stab}}{std\_pos + \epsilon} 
- W_{head} \cdot |mean\_head|
\right)
- 
W_{lat} (1 - r)
$$

Ý nghĩa:

- **tín hiệu ổn định hơn** → điểm cao  
- **góc hướng lệch nhiều** → bị phạt nhẹ  
- **vị trí r thấp (gần xe)** → ít nhiễu hơn  
- **coverage thấp** → điểm giảm mạnh (lane bị mất)


### 4) Ratio tối ưu

$$ 
r^* = \arg\max_{r} \; score(r)
$$

Ta lấy pos & head tương ứng $( r^* $) để đưa vào bộ điều khiển.

→ Kết quả: xe chạy mượt hơn, ổn định hơn, chống nhiễu tốt hơn.



In [ ]:
W_STAB, W_HEAD, W_LAT, EPS = 2.0, 0.1, 0.05, 1e-6

def adaptive_select_student(centers_px, pos_list, head_list, ratios, WIN = 15):
    """
    Student Version.
    Chỉ dùng dữ liệu đã tính từ Cell Multi-Ratio.
    """

    # TODO: tạo cửa sổ lịch sử đơn giản
    ### YOUR CODE HERE ###
    # hist_pos = ...
    # hist_head = ...
    # hist_ok = ...

    # TODO: tính score cho từng ratio
    scores = []
    for i, r in enumerate(ratios):
        ### YOUR CODE HERE ###
        # std_pos = ...
        # mean_head = ...
        # coverage = ...
        # score = ...
        scores.append(0)  # placeholder

    # TODO: chọn ratio tốt nhất
    ### YOUR CODE HERE ###
    # best_idx = ...

    return {
        "best_ratio": ratios[best_idx],
        "pos": pos_list[best_idx],
        "head": head_list[best_idx],
        "scores": scores
    }


In [ ]:
best_idx = select_best_ratio(pos_list, head_list, ratios)

print("Best Ratio      :", ratios[best_idx])
print("Selected pos_m  :", pos_list[best_idx])
print("Selected head   :", head_list[best_idx])


## 11. Tóm tắt

Trong lab này, bạn đã triển khai:

- Ước lượng độ rộng làn theo mét  
- Trích tâm làn từ mặt nạ BEV  
- Tính độ lệch ngang của làn (posₘ)
- Tính góc hướng (θ)
- Lấy mẫu đa tỉ lệ  
- Chọn thích ứng tỉ lệ tốt nhất  

Các đầu ra này được đưa trực tiếp vào:
  
**LAB 3.2 – Adaptive Steering Control + Video Demo**
